In [0]:
import numpy as np
import keras
from keras import Input, layers
from keras.layers import Dense, Embedding, LSTM, concatenate,  \
                         Flatten, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.models import Sequential, Model

import matplotlib.pyplot as plt
%matplotlib inline

## 1入力1出力

In [0]:
input_tensor = Input(shape=(32,))

In [0]:
dense = Dense(32, activation='relu')

In [0]:
output_tensor = dense(input_tensor)
output_tensor

In [0]:
# Sequentialモデルの書き方
model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(64,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [0]:
# Functional APIの書き方
input_tensor = Input(shape=(64,))
x = Dense(32, activation='relu')(input_tensor)
x = Dense(32, activation='relu')(x)
output_tensor = Dense(10, activation='softmax')(x)

model = Model(input_tensor, output_tensor)
model.summary()

In [0]:
# モデルをコンパイル
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [0]:
# 訓練に使用するダミーデータを作成
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))

# モデルを訓練
model.fit(x_train, y_train, epochs=10, batch_size=128)

# モデルを評価
score = model.evaluate(x_train, y_train)

## 多入力モデル

In [0]:
text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocaburaly_size = 500

In [0]:
# テキストに関するエンコード
text_input = Input(shape=(None,), dtype='int32', name='text')

# 入力をサイズが64のベクトルシーケンスに埋め込む
embedded_text = Embedding(text_vocabulary_size, 64)(text_input)

# LSTMを通じて単一のベクトルにエンコード
encoded_text = LSTM(32)(embedded_text)

In [0]:
# 質問に関するエンコード
question_input = Input(shape=(None,), dtype='int32', name='question')

embedded_question = Embedding(question_vocabulary_size, 32)(question_input)

encoded_question = LSTM(16)(embedded_question)

In [0]:
# エンコードされたテキストと質問を連結
concatenated = concatenate([encoded_text, encoded_question], axis=-1)

In [0]:
# ソフトマックス分類器を追加
answer = Dense(answer_vocaburaly_size, activation='softmax')(concatenated)

In [0]:
# モデルをインスタンス化
model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
num_samples = 10000
max_length = 100

In [0]:
# ダミーのNumpyデータを生成
text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))

answers = np.zeros(shape=(num_samples, answer_vocaburaly_size))
indices = np.random.randint(0, answer_vocaburaly_size, size=num_samples)
for i, x in enumerate(answers):
  x[indices[i]] = 1

In [0]:
# 入力リストを使った学習
model.fit([text, question], answers, epochs=10, batch_size=128)

In [0]:
# 入力ディクショナリを使った学習
model.fit({'text':text, 'question':question}, answers, epochs=10, batch_size=128)

## 多出力モデル

In [0]:
vocabulary_size = 50000
num_income_groups = 10

In [0]:
posts_input = Input(shape=(None, ), dtype='int32', name='posts')
embedded_posts = Embedding(256, vocabulary_size)(posts_input)
x = Conv1D(128, 5, activation='relu')(embedded_posts)
x = MaxPooling1D(5)(x)
x = Conv1D(256, 5, activation='relu')(x)
x = Conv1D(256, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(256, 5, activation='relu')(x)
x = Conv1D(256, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)

In [0]:
age_prediction = Dense(1, name='age')(x)
income_prediction = Dense(num_income_groups, activation='softmax', name='income')(x)
gender_prediction = Dense(1, activation='sigmoid', name='gender')(x)
model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])

In [0]:
model.compile(optimizer='rmsprop', 
              loss={'age': 'mse',
                    'income': 'categorical_crossentropy',
                    'gender': 'binary_crossentropy'},
              loss_weights={'age': 0.25,
                            'income': 1. ,
                            'gender': 10. })

In [0]:
# リスト型
model.fit(posts, 
          [age_targets, income_targets, gender_targets],
          epochs=10, 
          batchsize=64)

In [0]:
# 辞書型
model.fit(posts, 
          {'age': age_targets, 
           'income': income_targets, 
           'gender': gender_targets},
          epochs=10, 
          batch_size=64)

## マルチクラスの画像認識

In [0]:
batch_size = 50
num_classes1 = 10
num_classes2 = 2
epochs = 1

In [0]:
from keras.datasets import cifar10
(x_train,y_train),(x_test,y_test)=cifar10.load_data()

In [0]:
#画像を0-1の範囲で正規化
x_train=x_train.astype('float32')/255.0
x_test=x_test.astype('float32')/255.0

In [0]:
# 各ラベルに名前を設定
label1 = {0:"airplane",
          1:"automobile",
          2:"bird",
          3:"cat",
          4:"deer",
          5:"dog",
          6:"frog",
          7:"horse",
          8:"ship",
          9:"truck"}
label2 = {0:"artifact", #0,1,8,9
          1:"animal"}   #2,3,4,5,6,7

In [0]:
import copy

# 人工物を0、動物を1として、y_train2とy_test2を定義
def modLabel2(y):
    y2 = copy.deepcopy(y)
    for i in range(len(y)):
        if y2[i] in [0,1,8,9]:
            y2[i] = 0
        else:
            y2[i] = 1
    return y2

In [0]:
y_train2 = modLabel2(y_train)
y_test2 = modLabel2(y_test)

In [0]:
# クラスベクトルをバイナリクラス行列に変換
y_train1 = keras.utils.to_categorical(y_train, num_classes1)
y_test1 = keras.utils.to_categorical(y_test, num_classes1)

y_train2 = keras.utils.to_categorical(y_train2, num_classes2)
y_test2 = keras.utils.to_categorical(y_test2, num_classes2)
    
    
print("y train1:{0}\n y test1:{1}\n y train2:{2}\n y test2:{3}\n".format(
    y_train1.shape, y_test1.shape, y_train2.shape, y_test2.shape))

for i in range(10):
    plt.figure(figsize=(2, 2))
    print("data{0} is {1} and {2} ".format(
        i,label1[np.argmax(y_train1[i])], label2[np.argmax(y_train2[i])]))
    plt.imshow(x_train[i].reshape(32, 32, 3))
    plt.show()

In [0]:
from keras import layers
from keras import models
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten

In [0]:
# 入力層
model = models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'
          ,input_shape=x_train.shape[1:]))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

model_output = model.output

In [0]:
output1 = Dense(num_classes1, activation='softmax', name='output1')(model_output)
output2 = Dense(num_classes2, activation='softmax', name='output2')(model_output)

multiModel = Model(inputs=model.input, outputs=[output1, output2])

In [0]:
# Adamオプティマイザを設定
opt = keras.optimizers.adam(lr=0.0001, decay=1e-6)

# Adamを使ってモデルを訓練
multiModel.compile(loss={'output1': 'categorical_crossentropy',
                         'output2': 'categorical_crossentropy'},
                   optimizer=opt,
                   metrics=['accuracy'])

In [0]:
history = multiModel.fit(x_train,
                         {'output1': y_train1,
                          'output2': y_train2},
                         batch_size=batch_size,
                         epochs=epochs,
                         validation_data=(x_test,
                                          {'output1': y_test1,
                                           'output2': y_test2})
                         )

In [0]:
scores = multiModel.evaluate(x_test,
                            {'output1': y_test1,
                             'output2': y_test2}, batch_size=batch_size)

In [0]:
print("total loss:{0}".format(scores[0]))
print("label1 loss:{0}\taccuracy:{1}:".format(scores[1],scores[3]))
print("label2 loss:{0}\taccuracy:{1}:".format(scores[2],scores[4]))

In [0]:
model.summary()

### Q.Sequential モデルで記述されたモデルをFuntional APIを用いて書き直してください。